<a href="https://colab.research.google.com/github/KymieSaito/fine-tuning-llms-article/blob/main/Projeto2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <!-- Projeto 2:  -->
# <font color='yellow'>Projeto desenvolvido pela DSA, maiores informações:
https://www.datascienceacademy.com.br/course/ia-generativa-e-llms-para-processamento-de-linguagem-natural
1

</font>
## <font color='pink'>Projeto 2:  IA Generativa e LLMs Para Processamento de Linguagem Natural  (17/04/25) </font>

## Instalando e Carregando Pacotes

In [ ]:
!pip install -q -U watermark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.3 MB/s eta 0:00:00


In [ ]:
!pip install -q accelerate==1.4.0 peft==0.14.0 bitsandbytes==0.45.3 transformers==4.49.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 860.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
!pip install -q trl==0.15.2 gradio==5.18.0 protobuf==5.29.3 scipy==1.13.1

In [ ]:
!pip install -q sentencepiece==0.2.0 tokenizers==0.21.0 datasets==3.3.2

In [ ]:
%reload_ext watermark
%watermark -a "Desenvolvido por Kymie no curso da Data Science Academy"

Author: Desenvolvido por Kymie no curso da Data Science Academy



In [ ]:
# Imports
import os
import torch
import datasets
import pandas as pd
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          TrainingArguments,
                          pipeline,
                          logging)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Define o nível de log para CRITICAL
logging.set_verbosity(logging.CRITICAL)

In [ ]:
# Verifica o modelo da GPU
if torch.cuda.is_available():
    print('Número de GPUs:', torch.cuda.device_count())
    print('Modelo GPU:', torch.cuda.get_device_name(0))
    print('Total Memória [GB] da GPU:',torch.cuda.get_device_properties(0).total_memory / 1e9)

Número de GPUs: 1
Modelo GPU: Tesla T4
Total Memória [GB] da GPU: 15.828320256


In [ ]:
# Reset da memória da GPU
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
# Define o nome do dataset
nome_dataset = "dataset.csv"

In [ ]:
# Carrega os dados
dataset_carregado = load_dataset('csv', data_files = nome_dataset, delimiter = ',')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
Dados Carregados no Formato de Dicionário
dataset_carregado

DatasetDict({
    train: Dataset({
        features: ['train'],
        num_rows: 17057
    })
})

https://huggingface.co/NousResearch/Llama-2-7b-chat-hf

In [ ]:
# Nome do repositório do LLM pré-treinado
repositorio_hf = "NousResearch/Llama-2-7b-chat-hf"

In [ ]:
# Nome do novo modelo
modelo_Kymi = "novo_modelo_kymi"

## Definindo os Parâmetros de Configuração

In [ ]:
# Parâmetros LoRA
lora_r = 32
lora_alpha = 16
lora_dropout = 0.1

Os parâmetros acima são do LoRa (Low-Rank Adaptation), a parte base do QLoRA (Quantized Low-Rank Adaptation), uma técnica utilizada para adaptar modelos de linguagem de forma eficiente. Acima dos parâmetros LoRa colocamos os parâmetros de quantização, definindo assim o QLoRa.

Vamos descrever cada um dos parâmetros:

**lora_r**: Este parâmetro representa o "rank" na adaptação de Low-Rank (LoRA). Um valor de 32 significa que a matriz de pesos do modelo original será aproximada por duas matrizes menores cujo produto possui um rank máximo de 32. Essencialmente, isso reduz a complexidade computacional e o número de parâmetros a serem treinados durante a adaptação, mantendo a eficácia do modelo.

**lora_alpha**: Este é um fator de escala que é aplicado às atualizações de peso do LoRA durante o treinamento. Um valor de 16 indica que as atualizações de pesos serão escaladas por este fator. Esse parâmetro é importante porque permite um controle fino sobre a magnitude das atualizações dos pesos, o que pode afetar a rapidez e a eficácia da adaptação do modelo.

**lora_dropout**: Este parâmetro representa a taxa de "dropout" aplicada durante a adaptação do modelo. O valor 0.1 significa que 10% das unidades serão aleatoriamente descartadas (ou "desligadas") durante o treinamento. O dropout é uma técnica comum para evitar o overfitting em redes neurais, garantindo que o modelo não se torne excessivamente dependente de qualquer parte específica dos dados de treinamento.

Os parâmetros descritos abaixo referem-se ao LoRA (Low-Rank Adaptation), uma técnica eficiente para adaptar modelos de linguagem pré-treinados, ajustando apenas um subconjunto de parâmetros. Quando combinada com quantização, essa abordagem dá origem ao QLoRA (Quantized Low-Rank Adaptation), que reduz ainda mais os requisitos de memória e computação, mantendo a performance do modelo. Antes de configurar o QLoRA, definimos os parâmetros de quantização (como BitsAndBytesConfig, ajustado anteriormente) e, em seguida, os parâmetros do LoRA, que são detalhados a seguir.

Aqui está a descrição de cada parâmetro do LoRA:

    lora_r: Representa o "rank" (posto) na adaptação de Low-Rank (LoRA). Um valor de 32 indica que a matriz de pesos original do modelo será aproximada pelo produto de duas matrizes menores, cujo posto máximo é 32. Esse processo reduz a complexidade computacional e o número de parâmetros a serem treinados, permitindo uma adaptação eficiente sem comprometer significativamente a eficácia do modelo.
    lora_alpha: Refere-se a um fator de escala aplicado às atualizações de peso do LoRA durante o treinamento. Com um valor de 16, as atualizações de peso são multiplicadas por esse fator. Esse parâmetro permite ajustar a magnitude das atualizações, influenciando a velocidade e a estabilidade do processo de adaptação do modelo.
    lora_dropout: Define a taxa de dropout aplicada durante a adaptação do modelo. Um valor de 0.1 significa que 10% das unidades da rede serão descartadas aleatoriamente a cada iteração de treinamento. O dropout é uma técnica de regularização usada para prevenir o overfitting, garantindo que o modelo generalize melhor e não dependa excessivamente de padrões específicos dos dados de treinamento.

Explicação das correções

    Introdução confusa:
        A frase original "Os parâmetros acima são do LoRa [...] Acima dos parâmetros LoRa colocamos os parâmetros de quantização, definindo assim o QLoRa" estava invertida e pouco clara. Na verdade, no seu código (conversas anteriores), você primeiro definiu a quantização com BitsAndBytesConfig e depois trabalhou com os parâmetros do LoRA. Corrigi a ordem e expliquei que QLoRA é a combinação de LoRA com quantização.
    Coesão e clareza:
        Substituí "a parte base do QLoRA" por uma explicação mais precisa: LoRA é a técnica de adaptação, e QLoRA é a extensão que adiciona quantização.
        Usei "aqui está a descrição" em vez de "vamos descrever" para um tom mais direto e formal.
    Detalhes técnicos:
        Mantive as explicações dos parâmetros (lora_r, lora_alpha, lora_dropout), mas simplifiquei algumas frases para maior clareza. Por exemplo, "matriz de pesos do modelo original será aproximada por duas matrizes menores cujo produto possui um rank máximo de 32" foi ajustado para "cujo posto máximo é 32", mantendo a precisão técnica.
        Adicionei que o dropout ajuda na generalização, para reforçar o propósito do lora_dropout.

Contexto adicional para o seu projeto

Como você está trabalhando com fine-tuning de LLMs no Google Colab (conversas em 17/04/2025), esses parâmetros do LoRA provavelmente serão usados com uma biblioteca como peft (Parameter-Efficient Fine-Tuning), que implementa LoRA e QLoRA. Aqui está um exemplo de como configurar os parâmetros do LoRA no código, para complementar o texto:

In [ ]:
# Parâmetros bitsandbytes (QLoRa)
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

Os parâmetros acima são para a biblioteca bitsandbytes, uma ferramenta para otimização de treinamento de modelos de aprendizado de máquina, em particular para reduzir o uso de memória e acelerar o treinamento. Aqui está a explicação de cada parâmetro:

**use_4bit**: Este parâmetro indica se a quantização de 4 bits deve ser usada ou não. Ao definir True, isso significa que o modelo irá utilizar uma representação de 4 bits para os pesos durante o treinamento. Isso reduz significativamente a quantidade de memória necessária, permitindo treinar modelos maiores ou reduzir os requisitos de hardware.

**bnb_4bit_compute_dtype**: Este é o tipo de dado usado para cálculos durante o treinamento quando a quantização de 4 bits está ativa. O valor float16 significa que os cálculos serão feitos usando números de ponto flutuante de 16 bits. Isso é geralmente usado para equilibrar a eficiência computacional e a precisão numérica.

**bnb_4bit_quant_type**: Especifica o tipo de quantização a ser usado. O valor nf4 é um tipo específico de quantização desenvolvido pela bitsandbytes, otimizado para eficiência e eficácia em treinamento de modelos de aprendizado profundo. Este tipo de quantização é projetado para manter a precisão do modelo enquanto reduz os requisitos de memória.

**use_nested_quant**: Indica se uma técnica de quantização aninhada será usada. False significa que essa técnica não será empregada. A quantização aninhada pode ser usada para reduzir ainda mais o uso de memória, aplicando diferentes níveis de quantização a diferentes partes do modelo, mas pode ser mais complexa de implementar e gerenciar.

Esses parâmetros são usados para configurar como o modelo de aprendizado profundo irá lidar com a representação e cálculo dos pesos durante o treinamento, visando otimizar o uso de memória e acelerar o processo de treinamento.

In [ ]:
# Parâmetros do ajuste fino
output_dir = "saida"
num_train_epochs = 1
fp16 = True
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03

Os parâmetros acima são usados para configurar o processo de ajuste fino (fine-tuning) de modelos de aprendizado de máquina, especialmente modelos de linguagem natural. Vamos descrever cada um deles:

**output_dir**: Especifica o diretório onde os resultados do treinamento serão salvos.

**num_train_epochs**: O número de épocas de treinamento. O valor 1 significa que o modelo passará uma vez por todo o conjunto de dados de treinamento.

**fp16**: Indica se deve ser usado o treinamento com precisão mista de ponto flutuante de 16 bits (FP16). O valor True significa que sim, o que pode acelerar o treinamento e reduzir o uso de memória, mantendo uma precisão aceitável.

**bf16**: Semelhante ao fp16, mas para o formato bfloat16. False significa que não será utilizado. O formato bfloat16 é outra forma de reduzir o uso de memória e acelerar o treinamento, com impactos ligeiramente diferentes na precisão. Podemos usar fp16 ou bf16, mas não podemos usar ambos simultaneamente.

**per_device_train_batch_size**: Tamanho do lote de treinamento por dispositivo. O valor 4 indica que cada dispositivo de treinamento (como uma GPU) processará 4 exemplos por lote.

**per_device_eval_batch_size**: Tamanho do lote de avaliação por dispositivo, também definido como 4.

**gradient_accumulation_steps**: Número de passos para acumulação de gradientes antes de realizar uma atualização de parâmetros. O valor 1 significa que não há acumulação (cada passo resulta em uma atualização).

**gradient_checkpointing**: Habilita o checkpointing de gradientes, que é uma técnica para reduzir o uso de memória ao custo de um tempo de treinamento ligeiramente maior. True indica que está habilitado.

**max_grad_norm**: Norma máxima para o corte de gradientes. O valor 0.3 é um valor que ajuda a evitar o problema de explosão de gradientes em treinamentos.

**learning_rate**: Taxa de aprendizado inicial. O valor 2e-4 é um valor comum para ajuste fino, proporcionando um equilíbrio entre a velocidade de aprendizado e a estabilidade.

**weight_decay**: Taxa de decaimento de peso, usada para regularização. O valor 0.001 é um valor que ajuda a prevenir o overfitting.

**optim**: O otimizador usado. "paged_adamw_32bit" é uma variante do AdamW otimizado para eficiência em termos de memória.

**lr_scheduler_type**: Tipo de agendador de taxa de aprendizado. O valor "cosine" indica o uso do agendador cosseno, que ajusta a taxa de aprendizado seguindo uma curva cosseno.

**max_steps**: Número máximo de passos de treinamento. O valor -1 significa que o treinamento continuará até que o número de épocas seja alcançado.

**warmup_ratio**: Proporção do número total de passos de treinamento usados para o aquecimento linear da taxa de aprendizado. O valor 0.03 significa que 3% do treinamento inicial será usado para aumentar gradualmente a taxa de aprendizado.

Esses parâmetros são essenciais para configurar de maneira eficiente o processo de ajuste fino, impactando diretamente na qualidade do modelo treinado, no tempo de treinamento e no uso de recursos computacionais.

Os parâmetros abaixo são utilizados para configurar o processo de ajuste fino (fine-tuning) de modelos de aprendizado de máquina, com foco em modelos de linguagem natural (LLMs). Eles controlam aspectos como o uso de memória, a duração do treinamento e a estabilidade do modelo, impactando diretamente a qualidade do resultado, o tempo de treinamento e os recursos computacionais necessários. A seguir, detalhamos cada parâmetro:

    output_dir: Define o diretório onde os resultados do treinamento, como os pesos do modelo e os logs, serão salvos.
    num_train_epochs: Número de épocas de treinamento. Com o valor 1, o modelo percorre o conjunto de dados de treinamento uma única vez, o que pode ser suficiente para ajustes finos leves.
    fp16: Habilita o treinamento com precisão mista em ponto flutuante de 16 bits (FP16). O valor True indica que será usado, reduzindo o uso de memória e acelerando o treinamento, enquanto mantém uma precisão aceitável em GPUs compatíveis.
    bf16: Habilita o treinamento com o formato bfloat16, uma alternativa ao FP16 que preserva melhor o intervalo dinâmico dos valores. O valor False indica que não será usado. Não é possível usar fp16 e bf16 simultaneamente, devendo-se escolher um dos dois com base na compatibilidade do hardware e nas necessidades do modelo.
    per_device_train_batch_size: Tamanho do lote (batch size) para treinamento por dispositivo. O valor 4 significa que cada dispositivo (ex.: uma GPU) processará 4 exemplos por iteração durante o treinamento.
    per_device_eval_batch_size: Tamanho do lote para avaliação por dispositivo, também definido como 4, garantindo consistência entre as fases de treinamento e avaliação.
    gradient_accumulation_steps: Número de passos para acumulação de gradientes antes de atualizar os parâmetros do modelo. Com o valor 1, cada iteração de treinamento resulta em uma atualização imediata, sem acumulação.
    gradient_checkpointing: Ativa o checkpointing de gradientes, uma técnica que economiza memória ao custo de um leve aumento no tempo de treinamento. O valor True indica que está habilitado, sendo útil para modelos grandes em GPUs com memória limitada.
    max_grad_norm: Define a norma máxima para o corte (clipping) de gradientes, ajudando a evitar o problema de explosão de gradientes. O valor 0.3 é um limite conservador que promove estabilidade no treinamento.
    learning_rate: Taxa de aprendizado inicial. O valor 2e-4 (0.0002) é comumente usado em fine-tuning, equilibrando a velocidade de aprendizado e a estabilidade do modelo.
    weight_decay: Taxa de decaimento de peso, aplicada como regularização para prevenir overfitting. O valor 0.001 é pequeno, mas eficaz para evitar que o modelo se ajuste excessivamente aos dados de treinamento.
    optim: Tipo de otimizador utilizado. O valor "paged_adamw_32bit" refere-se a uma variante do otimizador AdamW otimizada para eficiência de memória, ideal para fine-tuning em hardware com recursos limitados.
    lr_scheduler_type: Tipo de agendador (scheduler) da taxa de aprendizado. O valor "cosine" indica o uso de um agendador cosseno, que ajusta a taxa de aprendizado suavemente ao longo do treinamento, seguindo uma curva cosseno, promovendo convergência mais estável.
    max_steps: Número máximo de passos de treinamento. O valor -1 desativa esse limite, permitindo que o treinamento continue até que o número de épocas definido em num_train_epochs seja atingido.
    warmup_ratio: Proporção do total de passos de treinamento usada para o aquecimento (warmup) linear da taxa de aprendizado. Com o valor 0.03, os primeiros 3% do treinamento são usados para aumentar gradualmente a taxa de aprendizado, o que ajuda na estabilidade inicial do treinamento.

Esses parâmetros são fundamentais para otimizar o fine-tuning de LLMs, permitindo ajustes precisos que equilibram desempenho, estabilidade e eficiência computacional.
Explicação das correções

    Introdução mais clara:
        A frase original "Os parâmetros acima são usados para configurar o processo de ajuste fino..." mencionava "acima" sem contexto, já que os parâmetros estão listados abaixo. Corrigi para "os parâmetros abaixo" e especifiquei que são usados para LLMs, alinhando com seu projeto.
    Precisão técnica:
        fp16 e bf16: Expliquei que não se pode usar ambos simultaneamente e que a escolha depende do hardware (ex.: GPUs NVIDIA mais recentes suportam melhor bf16).
        gradient_accumulation_steps: Simplifiquei a explicação, destacando que o valor 1 significa "sem acumulação".
        max_grad_norm: Adicionei que o valor 0.3 é conservador, para dar contexto sobre sua escolha.
        optim: Expliquei que "paged_adamw_32bit" é otimizado para eficiência de memória, o que é relevante para o seu ambiente no Colab.
    Coesão e estilo:
        Usei termos consistentes, como "taxa de aprendizado" em vez de alternar com "learning rate".
        Adicionei itálico (fine-tuning, batch size, etc.) para destacar termos técnicos, seguindo convenções de documentação técnica.
        Removi redundâncias, como "O valor 1 significa que..." em contextos óbvios.

Contexto adicional para o seu projeto

Como você está fazendo fine-tuning de LLMs no Google Colab (conversas em 17/04/2025 e 21/04/2025) e já enfrentou erros relacionados a memória de GPU, esses parâmetros são provavelmente usados com uma biblioteca como transformers da Hugging Face, possivelmente em conjunto com peft para QLoRA. Aqui está um exemplo de como configurar esses parâmetros em um script de treinamento:
training_config.py
python
Passos para usar no seu projeto

    Instale as dependências: No Google Colab, instale as bibliotecas necessárias:
    python

!pip install transformers
Use os parâmetros no treinamento: Após carregar o modelo com AutoModelForCausalLM (como corrigimos anteriormente), use o TrainingArguments com o Trainer:
python
from transformers import Trainer

trainer = Trainer(
    model=modelo,
    args=training_args,
    train_dataset=seu_dataset,  # Substitua pelo seu dataset
    eval_dataset=seu_dataset_eval  # Substitua pelo seu dataset de avaliação
)
trainer.train()
Documente no README: Adicione a explicação corrigida ao seu README.md no repositório:
markdown

    ### Parâmetros de Treinamento

    Os parâmetros abaixo são utilizados para configurar o processo de ajuste fino (*fine-tuning*) de modelos de linguagem natural (LLMs). Eles controlam aspectos como o uso de memória, a duração do treinamento e a estabilidade do modelo.

    - **output_dir**: Define o diretório onde os resultados do treinamento, como os pesos do modelo e os logs, serão salvos.
    - **num_train_epochs**: Número de épocas de treinamento. Com o valor 1, o modelo percorre o conjunto de dados de treinamento uma única vez.
    - **fp16**: Habilita o treinamento com precisão mista em ponto flutuante de 16 bits (FP16). O valor `True` indica que será usado, reduzindo o uso de memória e acelerando o treinamento.
    - **bf16**: Habilita o treinamento com o formato `bfloat16`. O valor `False` indica que não será usado.
    - **per_device_train_batch_size**: Tamanho do lote (*batch size*) para treinamento por dispositivo, definido como 4.
    - **per_device_eval_batch_size**: Tamanho do lote para avaliação por dispositivo, também definido como 4.
    - **gradient_accumulation_steps**: Número de passos para acumulação de gradientes. Com o valor 1, cada iteração de treinamento resulta em uma atualização imediata.
    - **gradient_checkpointing**: Ativa o *checkpointing* de gradientes, economizando memória. O valor `True` indica que está habilitado.
    - **max_grad_norm**: Define a norma máxima para o corte (*clipping*) de gradientes. O valor 0.3 promove estabilidade.
    - **learning_rate**: Taxa de aprendizado inicial. O valor 2e-4 (0.0002) equilibra velocidade e estabilidade.
    - **weight_decay**: Taxa de decaimento de peso para regularização. O valor 0.001 previne *overfitting*.
    - **optim**: Tipo de otimizador. O valor `"paged_adamw_32bit"` é otimizado para eficiência de memória.
    - **lr_scheduler_type**: Tipo de agendador da taxa de aprendizado. O valor `"cosine"` ajusta a taxa de aprendizado suavemente.
    - **max_steps**: Número máximo de passos de treinamento. O valor -1 desativa esse limite.
    - **warmup_ratio**: Proporção do total de passos usada para o *warmup*. O valor 0.03 indica 3% do treinamento inicial.

Resumo

    Correção do texto: Melhorei a introdução, corrigi imprecisões (ex.: relação entre fp16 e bf16), e simplifiquei explicações para maior clareza.
    Código complementar: Forneci um exemplo de TrainingArguments para usar no seu projeto.
    Integração no projeto: Mostrei como usar os parâmetros no treinamento e documentá-los no README.md.


In [ ]:
# Agrupando sequências em lotes de mesmo comprimento
group_by_length = True
save_steps = 0
logging_steps = 400

<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
Os parâmetros acima são usados para configurar certos aspectos do processo de treinamento de modelos de aprendizado de máquina, especialmente modelos de linguagem. Eles estão relacionados a como as sequências são agrupadas em lotes e como o progresso do treinamento é registrado e salvo. Vamos detalhar cada um:

**group_by_length**: Este parâmetro indica se as sequências devem ser agrupadas por comprimento ao formar lotes de treinamento. Quando True, isso significa que o treinamento agrupará sequências de comprimentos semelhantes juntas em um lote. Esta é uma prática eficiente porque reduz a quantidade de preenchimento (padding) necessário. O preenchimento é usado para garantir que todas as sequências em um lote tenham o mesmo comprimento, mas pode ser um desperdício de recursos computacionais. Agrupar sequências de comprimentos semelhantes minimiza esse desperdício.

**save_steps**: Especifica a frequência com que o modelo treinado deve ser salvo. Um valor de 0 indica que o modelo não será salvo automaticamente com base em um número de passos. Em vez disso, o modelo pode ser salvo no final de cada época de treinamento ou manualmente.

**logging_steps**: Define a frequência com que as informações de log devem ser registradas. O valor 400 significa que o processo de treinamento registrará informações como a perda de treinamento (loss), métricas de avaliação, entre outros, a cada 400 passos de treinamento. Isso é útil para monitorar o progresso do treinamento e para o ajuste fino dos hiperparâmetros.

In [ ]:
# Precisão dos dados para treinamento
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

O código acima está relacionado à configuração do tipo de dado (dtype) para computação durante o treinamento de modelos de rede neural usando a biblioteca PyTorch. Vamos analisar cada parte do código:

**torch**: É uma referência à biblioteca PyTorch, uma biblioteca para aprendizado de máquina e redes neurais.

**bnb_4bit_compute_dtype**: Esta é uma variável que armazena uma string representando o tipo de dado desejado para computação. O parâmetro bnb_4bit_compute_dtype foi definido como "float16", indicando que a computação deve ser realizada usando números de ponto flutuante de 16 bits.

**getattr**: É uma função Python built-in usada para obter um atributo de um objeto. Neste caso, ela está sendo usada para obter um atributo da biblioteca PyTorch com base no valor da string armazenada em bnb_4bit_compute_dtype.

O que acontece aqui é que getattr(torch, bnb_4bit_compute_dtype) recupera o tipo de dados de ponto flutuante de 16 bits (torch.float16) da biblioteca PyTorch, com base no valor de bnb_4bit_compute_dtype. Em seguida, esse tipo de dados é atribuído à variável compute_dtype.

O uso de compute_dtype no treinamento de modelos de rede neural tem implicações importantes:

**Eficiência de Memória**: Usar float16 ao invés de tipos de dados mais comuns como float32 pode reduzir significativamente o uso de memória, permitindo o treinamento de modelos maiores ou a execução de mais processos em paralelo.

**Velocidade de Computação**: Muitas GPUs modernas têm otimizações para cálculos float16, o que pode acelerar o treinamento.

**Precisão**: Embora float16 possa ser menos preciso do que float32, muitas vezes é suficientemente preciso para tarefas de treinamento de modelos de rede neural.


# Importando as bibliotecas necessárias
import torch
from transformers import BitsAndBytesConfig

# Definindo os parâmetros para a quantização
use_4bit = True  # Ativa a quantização em 4 bits para reduzir uso de memória
bnb_4bit_quant_type = "nf4"  # Tipo de quantização: "nf4" (normal float 4) é mais estável
compute_dtype = torch.float16  # Tipo de dado para cálculos, eficiente em GPUs
use_nested_quant = False  # Quantização dupla, geralmente desativada para simplicidade

# Configurando a quantização com BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant
)


In [ ]:
# Definindo os parâmetros da quantização
bnb_config = BitsAndBytesConfig(load_in_4bit = use_4bit,
                                bnb_4bit_quant_type = bnb_4bit_quant_type,
                                bnb_4bit_compute_dtype = compute_dtype,
                                bnb_4bit_use_double_quant = use_nested_quant)

In [ ]:
# Configuração para 4-bit com offload
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    llm_int8_enable_fp32_cpu_offload=True
)

# Carregando o modelo com o device_map personalizado
modelo = AutoModelForCausalLM.from_pretrained(
    repositorio_hf,
    quantization_config=bnb_config,
    device_map="auto"
)

# Carregando o tokenizer
tokenizer = AutoTokenizer.from_pretrained(repositorio_hf)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Não usaremos o cache
modelo.config.use_cache = False
modelo.config.pretraining_tp = 1

In [ ]:
# Carregando o tokenizador do modelo base
tokenizador = AutoTokenizer.from_pretrained(repositorio_hf, trust_remote_code = True)
tokenizador.pad_token = tokenizador.eos_token
tokenizador.padding_side = "right"

In [ ]:
# Carregando a configuração LoRA
peft_config = LoraConfig(lora_alpha = lora_alpha,
                         lora_dropout = lora_dropout,
                         r = lora_r,
                         bias = "none",
                         task_type = "CAUSAL_LM")

https://github.com/huggingface/peft/blob/main/src/peft/tuners/lora/config.py

In [ ]:
# Definindo os parâmetros de treino
training_arguments = TrainingArguments(output_dir = output_dir,
                                       num_train_epochs = num_train_epochs,
                                       per_device_train_batch_size = per_device_train_batch_size,
                                       gradient_accumulation_steps = gradient_accumulation_steps,
                                       optim = optim,
                                       save_steps = save_steps,
                                       logging_steps = logging_steps,
                                       learning_rate = learning_rate,
                                       weight_decay = weight_decay,
                                       fp16 = fp16,
                                       bf16 = bf16,
                                       max_grad_norm = max_grad_norm,
                                       max_steps = max_steps,
                                       warmup_ratio = warmup_ratio,
                                       group_by_length = group_by_length,
                                       lr_scheduler_type = lr_scheduler_type)

Veja a descrição de Supervised Fine-Tuning e RLHF no videobook.

In [ ]:
# Definindo os Parâmetros do Fine-Tuning Supervisionado
sft_config = SFTConfig(per_device_train_batch_size = 2,
                       gradient_accumulation_steps = 4,
                       warmup_steps = 5,
                       num_train_epochs = 1,
                       learning_rate = 2e-4,
                       fp16 = True,
                       logging_steps = 1,
                       optim = "adamw_8bit",
                       weight_decay = 0.01,
                       lr_scheduler_type = "linear",
                       seed = 3407,
                       output_dir = "outputs",
                       report_to = "none",
                       max_seq_length = 2048,
                       dataset_text_field = "train",
                       packing = False)

In [ ]:
# Definindo os Parâmetros do Fine-Tuning Supervisionado (requer os parâmetros gerais no sft_config acima)
Kymi_trainer = SFTTrainer(model = modelo,
                         train_dataset = dataset_carregado['train'],
                         peft_config = peft_config,
                         tokenizer = tokenizador,
                         args = sft_config)

https://huggingface.co/docs/trl/en/sft_trainer

> Treinamento do Modelo com o Ajuste Fino

In [ ]:
%%time
Kymi_trainer.train()

ValueError: You can't train a model that has been loaded in 8-bit or 4-bit precision on a different device than the one you're training on. Make sure you loaded the model on the correct device using for example `device_map={'':torch.cuda.current_device()}` or `device_map={'':torch.xpu.current_device()}`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
print(torch.cuda.is_available())  # deve retornar True
print(torch.cuda.get_device_name(0))  # deve mostrar o nome da sua GPU


In [ ]:
# Salvando o modelo treinado
Kymi_trainer.model.save_pretrained(modelo_Kymi)

<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->

In [ ]:
# Novo texto de entrada
prompt = "It's rare that a movie lives up to its hype, even rarer that the hype is transcended by the actual achievement"

https://huggingface.co/docs/transformers/en/main_classes/pipelines

In [ ]:
# Pipeline de Análise de Sentimentos com o Modelo Ajustado
pipe = pipeline(task = "text-generation",
                model = modelo,
                tokenizer = tokenizador,
                max_length = 200)

In [ ]:
# Executa o pipeline e extrai o resultado
resultado = pipe(f"<s>[INST] {prompt} [/INST]")

AssertionError: 

In [ ]:
print(resultado)

NameError: name 'resultado' is not defined

In [ ]:
print(resultado[0]['generated_text'])

NameError: name 'resultado' is not defined

In [ ]:
# Libera a memória
del modelo
del pipe
del dsa_trainer
import gc
gc.collect()

220

In [ ]:
# Carrega o modelo em fp16 e faz o merge com os pesos LoRA
base_model = AutoModelForCausalLM.from_pretrained(repositorio_hf,
                                                  low_cpu_mem_usage = True,
                                                  return_dict = True,
                                                  torch_dtype = torch.float16,
                                                  device_map = "auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Cria o modelo final
modelo_dsa_final = PeftModel.from_pretrained(base_model, modelo_dsa)

In [ ]:
# Faz o merge e descarrega o modelo
modelo_dsa_final = modelo_dsa_final.merge_and_unload()

In [ ]:
# Carrega o tokenizador
tokenizador_dsa = AutoTokenizer.from_pretrained(repositorio_hf, trust_remote_code = True)
tokenizador_dsa.pad_token = tokenizador_dsa.eos_token
tokenizador_dsa.padding_side = "right"

In [ ]:
# Salva modelo e tokenizador
modelo_dsa_final.save_pretrained('novo_modelo-dsa-llm-projeto2')
tokenizador_dsa.save_pretrained('novo_modelo-dsa-llm-projeto2')

Saving checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Novo texto de entrada
prompt = "It's rare that a movie lives up to its hype, even rarer that the hype is transcended by the actual achievement"

In [ ]:
# Cria o pipeline
pipe = pipeline(task = "text-generation",
                model = modelo_dsa_final,
                tokenizer = tokenizador_dsa,
                max_length = 200)

In [ ]:
# Executa o pipeline e extrai o resultado
resultado = pipe(f"<s>[INST] {prompt} [/INST]")

In [ ]:
print(resultado)

In [ ]:
# Vamos não apenas classificar o sentimento.
# Vamos gerar texto positivo e/ou negativo a partir da avaliação (texto) inicial.
print(resultado[0]['generated_text'])

In [ ]:
# Libera a memória da GPU
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
%watermark -a "Data Science Academy"

In [ ]:
#%watermark -v -m

In [ ]:
#%watermark --iversions

# Fim